In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, Bidirectional, LSTM
import feature_builder
import util

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [4]:
processed, X_train = prepare_for_cnn(train_df)
y_train = train_df['target'].values

Percentage of words covered in the embeddings = 0.6336399642263958


In [5]:
pd.set_option('display.max_colwidth', -1)
processed

c:\users\maxi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [14]:
def RNN():
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1)
    ])
    return model

Compilo ni modelo

In [15]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_6 (Bidirection (None, 1, 128)            202752    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 248,193
Trainable params: 248,193
Non-trainable params: 0
_________________________________________________________________


Entrenamos

In [16]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=8)
history = model.fit(X_train, y_train, epochs=30, shuffle=False, validation_split=0.0)#, callbacks=[es])
util.plot_history(history)

Epoch 1/30
238/238 [==============================] - 1s 3ms/step - loss: 0.6827 - accuracy: 0.5703
Epoch 2/30
238/238 [==============================] - 1s 3ms/step - loss: 0.6626 - accuracy: 0.5703
Epoch 3/30
238/238 [==============================] - 1s 3ms/step - loss: 0.6361 - accuracy: 0.5736
Epoch 4/30
238/238 [==============================] - 1s 3ms/step - loss: 0.6193 - accuracy: 0.5990
Epoch 5/30
238/238 [==============================] - 1s 3ms/step - loss: 0.6058 - accuracy: 0.6321
Epoch 6/30
238/238 [==============================] - 1s 3ms/step - loss: 0.5855 - accuracy: 0.6647
Epoch 7/30
238/238 [==============================] - 1s 3ms/step - loss: 0.5660 - accuracy: 0.6978
Epoch 8/30
238/238 [==============================] - 1s 3ms/step - loss: 0.5464 - accuracy: 0.7175
Epoch 9/30
238/238 [==============================] - 1s 3ms/step - loss: 0.5278 - accuracy: 0.7370
Epoch 10/30
238/238 [==============================] - 1s 3ms/step - loss: 0.5137 - accuracy: 0.7519

KeyError: 'val_accuracy'

In [17]:
ids = test_df['id']

processed, final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

Percentage of words covered in the embeddings = 0.6973331905322909
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


,target
id,
0,1
2,1
3,1
9,1
11,1
...,...
10861,0
10865,1
10868,1


In [18]:
final_df.to_csv('lstm.csv')

In [19]:
final_df['target'].value_counts()

0    2122
1    1141
Name: target, dtype: int64